In [2]:
from numpy import mean
from numpy import std
from numpy import dstack
from pandas import read_csv
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Dropout
from keras.layers import LSTM
from keras.layers import LSTM, Conv1D, concatenate,GlobalMaxPooling1D,TimeDistributed, MaxPooling1D
from tensorflow.keras.utils import to_categorical
from matplotlib import pyplot
import numpy as np 
from numpy import save, load

In [3]:
def data_normalize(data_temp):
    data_temp2=data_temp.T
    data_temp2 -=np.mean(data_temp2,axis=0)
    data_temp2 /=np.std(data_temp2,axis=0)
    data_temp=data_temp2.T
    return data_temp

In [4]:
def load_file(filepath):
    dataframe = read_csv(filepath, header=None, delim_whitespace=True)
    return dataframe.values

In [5]:
def load_group(filenames, prefix=''):
    loaded = list()
    for name in filenames:
        data = load_file(prefix + name)
        loaded.append(data)
    # stack group so that features are the 3rd dimension
    loaded = dstack(loaded)
    return loaded

In [6]:
def load_dataset_group(group, prefix=''):
    filepath = prefix + group + '/data/'
    # load all 9 files as a single array
    filenames = list()
    # body acceleration
    filenames += ["acc_x.txt","acc_y.txt","acc_z.txt"]
    # body gyroscope
    filenames += ["gyr_x.txt","gyr_y.txt","gyr_z.txt"]
    # load input data
    X = load_group(filenames, filepath)
    # load class output
    y = load_file(prefix + group + '/y_'+group+'.txt')
    return X, y

In [7]:
def load_dataset(prefix=''):
    # load all train
    trainX, trainy = load_dataset_group('train', prefix + 'Dataset_5/')
    print(trainX.shape, trainy.shape)
    # load all test
    testX, testy = load_dataset_group('test', prefix + 'Dataset_5/')
    print(testX.shape, testy.shape)
    # zero-offset class values
    trainy = trainy - 1
    testy = testy - 1
    # one hot encode y
    trainy = to_categorical(trainy)
    testy = to_categorical(testy)
    print(trainX.shape, trainy.shape, testX.shape, testy.shape)
    return trainX, trainy, testX, testy

In [8]:
trainX, trainy, testX, testy=load_dataset()

(66542, 256, 6) (66542, 1)
(7600, 256, 6) (7600, 1)
(66542, 256, 6) (66542, 2) (7600, 256, 6) (7600, 2)


In [9]:
trainX.transpose(0,2,1)

array([[[-1.20189e+01, -1.20919e+01, -1.16355e+01, ..., -6.71162e+00,
         -6.97326e+00, -7.22548e+00],
        [-1.75256e+00, -1.21401e+00, -5.71316e-01, ..., -3.42565e+00,
         -3.13910e+00, -3.10191e+00],
        [-4.77883e+00, -4.23115e+00, -4.08271e+00, ..., -2.47852e+00,
         -1.93092e+00, -1.33028e+00],
        [ 3.90950e-02,  2.34725e-01,  1.98951e-01, ..., -7.84141e-01,
         -4.38391e-01, -7.88780e-02],
        [ 6.84170e-02,  3.11540e-02, -1.71810e-02, ..., -5.16563e-01,
         -2.85809e-01, -7.93550e-02],
        [-1.08734e-01,  9.73760e-02,  2.85770e-01, ..., -2.85618e-01,
         -1.33837e-01, -6.21170e-02]],

       [[-1.33596e+01, -1.33508e+01, -1.22107e+01, ..., -3.69784e+00,
         -2.93828e+00, -1.74657e+00],
        [-1.03430e+00, -5.13407e-01, -5.90470e-01, ...,  1.81570e+00,
          3.47339e+00,  7.46856e+00],
        [-3.96480e+00, -3.90300e+00, -4.56844e+00, ...,  4.84048e+00,
          9.57082e+00,  1.33519e+01],
        [ 2.78555e-01,  2.

In [10]:
data_normalize(trainX)

array([[[-2.05684166,  0.30859801, -0.38867668,  0.7214084 ,
          0.7281644 ,  0.68734752],
        [-2.09457797,  0.3740654 , -0.31064852,  0.70284332,
          0.65664463,  0.67167314],
        [-2.09419714,  0.48074051, -0.33645673,  0.66000267,
          0.60970286,  0.68020782],
        ...,
        [-1.93426052, -0.47131339, -0.04964152,  0.70471317,
          0.82384161,  0.92666066],
        [-2.00381402, -0.41084718,  0.0911117 ,  0.7112082 ,
          0.77460098,  0.83774033],
        [-2.02913662, -0.43409469,  0.25119115,  0.73524711,
          0.7350626 ,  0.74173045]],

       [[-2.13069139,  0.39192567, -0.20785927,  0.66062748,
          0.64612437,  0.63987314],
        [-2.1362447 ,  0.48327962, -0.20838116,  0.63729853,
          0.56630276,  0.65774497],
        [-2.08466594,  0.52875987, -0.36589794,  0.63538643,
          0.57411248,  0.71230509],
        ...,
        [-1.72216807,  0.45365923,  1.6473386 ,  0.07699147,
         -0.27685858, -0.17896265],
  

In [11]:
# save numpy array as npy file
from numpy import asarray
from numpy import save
# define data
#data = asarray([[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]])
# save to npy file
save('trainX.npy', trainX)
#np.load(tranx.npy)
# load numpy array from npy file
from numpy import load
# load array
trainX = load('trainX.npy')
# print the array
print(trainX)

[[[-2.05684166  0.30859801 -0.38867668  0.7214084   0.7281644
    0.68734752]
  [-2.09457797  0.3740654  -0.31064852  0.70284332  0.65664463
    0.67167314]
  [-2.09419714  0.48074051 -0.33645673  0.66000267  0.60970286
    0.68020782]
  ...
  [-1.93426052 -0.47131339 -0.04964152  0.70471317  0.82384161
    0.92666066]
  [-2.00381402 -0.41084718  0.0911117   0.7112082   0.77460098
    0.83774033]
  [-2.02913662 -0.43409469  0.25119115  0.73524711  0.7350626
    0.74173045]]

 [[-2.13069139  0.39192567 -0.20785927  0.66062748  0.64612437
    0.63987314]
  [-2.1362447   0.48327962 -0.20838116  0.63729853  0.56630276
    0.65774497]
  [-2.08466594  0.52875987 -0.36589794  0.63538643  0.57411248
    0.71230509]
  ...
  [-1.72216807  0.45365923  1.6473386   0.07699147 -0.27685858
   -0.17896265]
  [-1.2418291   0.39643237  1.95440161 -0.15229338 -0.48769639
   -0.46901511]
  [-0.95046994  0.76923106  1.86716301 -0.34494964 -0.68623857
   -0.65473592]]

 [[-1.95658954  0.21439491 -0.57505151

In [12]:
# save numpy array as npy file
from numpy import asarray
from numpy import save
# define data
#data = asarray([[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]])
# save to npy file
save('trainy.npy', trainy)
#np.load(tranx.npy)
# load numpy array from npy file
from numpy import load
# load array
trainy = load('trainy.npy')
# print the array
print(trainy)

[[1. 0.]
 [0. 1.]
 [1. 0.]
 ...
 [0. 1.]
 [1. 0.]
 [0. 1.]]


In [13]:
data_normalize(testX)

array([[[-1.94144189,  0.6035432 ,  1.35293101,  0.04212684,
         -0.15646318,  0.09930402],
        [-1.98568433,  0.58576818,  1.29978902,  0.09998684,
         -0.08503135,  0.08517163],
        [-2.0788719 ,  0.52631715,  1.15320014,  0.2410066 ,
          0.05929839,  0.09904963],
        ...,
        [-2.20134391,  0.08006604,  0.45731217,  0.46663196,
          0.64333559,  0.55399816],
        [-2.20710268,  0.17671052,  0.72952861,  0.42629981,
          0.46069918,  0.41386457],
        [-2.14381375,  0.30235743,  1.04649856,  0.33201471,
          0.22471015,  0.23823291]],

       [[-2.00132021,  0.39432574,  1.30839812,  0.26319787,
         -0.15174003,  0.18713851],
        [-2.10633204,  0.35431243,  1.12780409,  0.31917369,
          0.05900927,  0.24603257],
        [-2.17737243,  0.26543255,  0.92656302,  0.39275913,
          0.28586352,  0.30675421],
        ...,
        [ 1.71520309, -1.25436974,  0.88145648, -0.18131117,
         -0.55975342, -0.60122523],
  

In [14]:
# save numpy array as npy file
from numpy import asarray
from numpy import save
# define data
#data = asarray([[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]])
# save to npy file
save('testX.npy', testX)
#np.load(tranx.npy)
# load numpy array from npy file
from numpy import load
# load array
testX = load('testX.npy')
# print the array
print(testX)

[[[-1.94144189  0.6035432   1.35293101  0.04212684 -0.15646318
    0.09930402]
  [-1.98568433  0.58576818  1.29978902  0.09998684 -0.08503135
    0.08517163]
  [-2.0788719   0.52631715  1.15320014  0.2410066   0.05929839
    0.09904963]
  ...
  [-2.20134391  0.08006604  0.45731217  0.46663196  0.64333559
    0.55399816]
  [-2.20710268  0.17671052  0.72952861  0.42629981  0.46069918
    0.41386457]
  [-2.14381375  0.30235743  1.04649856  0.33201471  0.22471015
    0.23823291]]

 [[-2.00132021  0.39432574  1.30839812  0.26319787 -0.15174003
    0.18713851]
  [-2.10633204  0.35431243  1.12780409  0.31917369  0.05900927
    0.24603257]
  [-2.17737243  0.26543255  0.92656302  0.39275913  0.28586352
    0.30675421]
  ...
  [ 1.71520309 -1.25436974  0.88145648 -0.18131117 -0.55975342
   -0.60122523]
  [ 1.79754931 -1.16747328  0.78191766 -0.1891264  -0.68544847
   -0.53741882]
  [ 1.33283304 -1.62305497  1.11584683 -0.02913977 -0.51245965
   -0.28402549]]

 [[-2.01902418  0.20974278  1.327032

In [15]:
# save numpy array as npy file
from numpy import asarray
from numpy import save
# define data
#data = asarray([[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]])
# save to npy file
save('testy.npy', testy)
#np.load(tranx.npy)
# load numpy array from npy file
from numpy import load
# load array
testy = load('testy.npy')
# print the array
print(testy)

[[1. 0.]
 [0. 1.]
 [1. 0.]
 ...
 [0. 1.]
 [1. 0.]
 [0. 1.]]


In [16]:
X_train = load('trainX.npy')
X_test = load('testX.npy')
y_train = load('trainy.npy')
y_test = load('testy.npy')
print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)

(66542, 256, 6) (66542, 2) (7600, 256, 6) (7600, 2)


In [17]:
from sklearn.model_selection import train_test_split
X_training, X_validation, y_training, y_validation = train_test_split(X_train, y_train, test_size=0.20, random_state=42)

In [18]:
n_timesteps, n_features, n_outputs = X_train.shape[1], X_train.shape[2], y_train.shape[1]

In [19]:
model = Sequential()
model.add(Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=(n_timesteps,n_features)))
model.add(Conv1D(filters=64, kernel_size=3, activation='relu'))
model.add(Dropout(0.5))
model.add(MaxPooling1D(pool_size=2))
model.add(Flatten())
model.add(Dense(100, activation='relu'))
model.add(Dense(n_outputs, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d (Conv1D)              (None, 254, 64)           1216      
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 252, 64)           12352     
_________________________________________________________________
dropout (Dropout)            (None, 252, 64)           0         
_________________________________________________________________
max_pooling1d (MaxPooling1D) (None, 126, 64)           0         
_________________________________________________________________
flatten (Flatten)            (None, 8064)              0         
_________________________________________________________________
dense (Dense)                (None, 100)               806500    
_________________________________________________________________
dense_1 (Dense)              (None, 2)                 2

In [20]:
history = model.fit(X_training, y_training, epochs=10, verbose=False, validation_data=(X_validation, y_validation), batch_size=100)

In [21]:
loss, accuracy = model.evaluate(X_validation, y_validation, verbose=False)
print("Validation Accuracy:  {:.4f}".format(accuracy))
loss, accuracy = model.evaluate(X_test, y_test, verbose=False)
print("Testing Accuracy: {:.4f}".format(accuracy))

Validation Accuracy:  0.9557
Testing Accuracy: 0.8875
